# Part 3 of the IBM Capstone Project: Segmenting and Clustering Neighborhoods in Toronto

By Tobias Mehra

## Part 1: Library Installation 

In [1]:
# import all the necessary libraries


import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
# Loading the Toronto file that includes the coordinates with pandas read function

df_toronto = pd.read_csv("toronto_geo.csv")

In [3]:
df_toronto.head(20)

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,1,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450
2,2,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
3,3,M1W,Scarborough,Steeles West,43.799525,-79.318389
4,4,M9W,Etobicoke,Northwest,43.706748,-79.594054
5,5,M9B,Etobicoke,"Islington, Princess Gardens, West Deane Park",43.650943,-79.554724
6,6,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.394420
7,7,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
8,8,M3A,North York,Parkwoods,43.753259,-79.329656
9,9,M6P,West Toronto,High Park,43.661608,-79.464763


## Part 2: Creating a map of Toronto 

In [4]:
# As found on google.com 
latitude = 43.6532
longitude = - 79.3832

toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map


## Part 3: Let's Explore A Borough: Downtown

In [5]:
downtown_data = df_toronto[df_toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head(10)

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,6,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.394420
1,10,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
2,13,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,24,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049
4,25,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
5,27,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
6,33,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
7,37,M5J,Downtown Toronto,"Toronto Islands, Union Station",43.640816,-79.381752
8,41,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
9,45,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280


### Let's Create A map of Downtown Toronto

In [6]:
# as found on google.com 
latitude_downtown = 43.6543
longitude_downtown = -79.3860

map_downtown = folium.Map(location=[latitude_downtown, longitude_downtown], zoom_start=12)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_downtown)  
    
map_downtown

## Part 4: Let's explore a neighbourhood in Downtown Toronto

In [7]:
CLIENT_ID = 'XFAQWDOTBHM21V31EO0GLP1BZWMPXEOUWS3UCHJIPRWBCZ4K' # your Foursquare ID
CLIENT_SECRET = 'LRODUIDEQKZALMTF5G3NZJTYFP2OFW5FDYK4EJSF4AIEZVEV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XFAQWDOTBHM21V31EO0GLP1BZWMPXEOUWS3UCHJIPRWBCZ4K
CLIENT_SECRET:LRODUIDEQKZALMTF5G3NZJTYFP2OFW5FDYK4EJSF4AIEZVEV


In [8]:
# let's explore first neighbourhood in North York Dataframe

downtown_data.loc[5, 'Neighbourhood']

'Ryerson'

In [9]:
neighborhood_latitude = downtown_data.loc[5, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[5, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[5, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Ryerson are 43.6571618, -79.37893709999999.


#### Now, let's get the top 100 venues that are in Ryerson within a radius of 500 meters.

In [10]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c5a3291351e3d1df7c7a8a1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Toronto',
  'headerFullLocation': 'Downtown Toronto, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 129,
  'suggestedBounds': {'ne': {'lat': 43.6616618045, 'lng': -79.37272880013877},
   'sw': {'lat': 43.652661795499995, 'lng': -79.3851453998612}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57eda381498ebe0e6ef40972',
       'name': 'UNIQLO ユニクロ',
       'location': {'address': '220 Yonge St',
        'crossStreet': 'at Dundas St W',
        'lat': 43.65591027779457,
        'lng': -79.38064099181345,
        'labeledLatLngs': [{'

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641
1,Silver Snail Comics,Comic Shop,43.657031,-79.381403
2,Burrito Boyz,Burrito Place,43.656265,-79.378343
3,Blaze Pizza,Pizza Place,43.656518,-79.380015
4,Tacos 101,Taco Place,43.656636,-79.376968
5,Yonge-Dundas Square,Plaza,43.656125,-79.380622
6,Page One Cafe,Café,43.657772,-79.376073
7,Hokkaido Ramen Santouka らーめん山頭火,Ramen Restaurant,43.656435,-79.377586
8,DAVIDsTEA,Tea Room,43.656586,-79.381167
9,Ed Mirvish Theatre,Theater,43.655102,-79.379768


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## Let's create a function to apply this to all neighbourhoods in downtown Toronto (as seen on Coursera)

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's write code to run the above function on each neighborhood and create a new dataframe called downtown_venues

In [16]:
# get venues for each neighbourhood in downtown toronto

downtown_venues = getNearbyVenues(names=downtown_data['Neighbourhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude'])



CN Tower, King and Spadina, Railway Lands, South Niagara
Design Exchange, Toronto Dominion Centre
Church and Wellesley
Chinatown, Grange Park, Kensington Market
Harbourfront, Regent Park
Ryerson
Cabbagetown, St. James Town
Toronto Islands, Union Station
St. James Town
First Canadian Place, Underground city
Rosedale
Commerce Court
University of Toronto
Adelaide, King
Berczy Park


Let's check the size of the dataframe

In [17]:
print(downtown_venues.shape)
downtown_venues.head(10)

(1089, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.39442,Billy Bishop Toronto City Airport (YTZ) (Billy...,43.631585,-79.395643,Airport
1,"CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.39442,Porter Lounge,43.630680,-79.395756,Airport Lounge
2,"CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.39442,Toronto Harbour,43.633045,-79.396484,Harbor / Marina
3,"CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.39442,Billy Bishop Café,43.631132,-79.396139,Airport Food Court
4,"CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.39442,Air Canada Check-In Counter,43.631226,-79.395987,Airport Terminal
5,"CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.39442,Gate 8,43.631536,-79.394570,Airport Gate
6,"CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.39442,Crew Room,43.631360,-79.396107,Airport Lounge
7,"CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.39442,Porter Airlines Flight,43.631734,-79.394582,Plane
8,"CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.39442,Want Passport,43.631483,-79.396077,Boutique
9,"CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.39442,Porter FBO Limited,43.630717,-79.398698,Airport Terminal


Let's use the count function to find out how many venues there are by neighbourhood


In [18]:
downtown_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"CN Tower, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",47,47,47,47,47,47
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Church and Wellesley,86,86,86,86,86,86
Commerce Court,100,100,100,100,100,100
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100


In [19]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 199 uniques categories.


## Analysis of each neighbourhood in downtown Toronto

In [20]:

# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"CN Tower, King and Spadina, Railway Lands, Sou...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"CN Tower, King and Spadina, Railway Lands, Sou...",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"CN Tower, King and Spadina, Railway Lands, Sou...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [21]:
downtown_onehot.shape

(1089, 200)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_grouped

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.010000,0.01,0.000000,0.03,0.00,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.020000,0.010000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.00,0.030000,0.010000,0.060000,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.020000,0.000000,0.000000,0.010000,0.01,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.01,0.00,0.000000,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.01,0.00,0.000000,0.01,0.00,0.010000,0.000000,0.030000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.030000,0.00,0.00,0.010000,0.010000,0.00,0.000000,0.000000,0.020000,0.010000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.00,0.010000,0.000000,0.000000,0.00,0.00,0.01,0.00,0.01,0.00,0.000000,0.00,0.01,0.01,0.000000,0.010000,0.01,0.01,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.01,0.00

In [23]:
downtown_grouped.shape

(15, 200)

#### Let's print each neighborhood along with the top 5 most common venues

In [24]:
#top 5 in each neighbourhood

num_top_venues = 5

for hood in downtown_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1          Restaurant  0.05
2        Cocktail Bar  0.05
3  Italian Restaurant  0.04
4              Bakery  0.04


----CN Tower, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3     Boat or Ferry  0.07
4           Airport  0.07


----Cabbagetown, St. James Town----
         venue  freq
0  Coffee Shop  0.09
1         Park  0.06
2   Restaurant  0.06
3         Café  0.04
4  Pizza Place  0.04


----Chinatown, Grange Park, Kensington Market----
                           venue  freq
0                           Café  0.07
1                            Bar  0.07
2          Vietnamese Restaurant  0.05
3  Vegetarian / Ve

#### Let's put that into a *pandas* dataframe

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Clothing Store,Asian Restaurant,Gym,Bakery,Restaurant
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Beer Bar,Seafood Restaurant,Pub,Italian Restaurant,Café,Farmers Market,Bakery
2,"CN Tower, King and Spadina, Railway Lands, Sou...",Airport Lounge,Airport Terminal,Airport Service,Boutique,Plane,Sculpture Garden,Boat or Ferry,Airport Gate,Airport Food Court,Airport
3,"Cabbagetown, St. James Town",Coffee Shop,Park,Restaurant,Café,Pub,Bakery,Pizza Place,Italian Restaurant,Japanese Restaurant,Beer Store
4,"Chinatown, Grange Park, Kensington Market",Bar,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Chinese Restaurant,Gaming Cafe


## Let's Cluster all neighbourhoods in Downtown Toronto using k-means

In [27]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 2, 0, 4, 4, 4, 4, 4, 0], dtype=int32)

In [28]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtown_merged.head() # check the last columns!

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,6,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Sou...",43.628947,-79.394420,2,Airport Lounge,Airport Terminal,Airport Service,Boutique,Plane,Sculpture Garden,Boat or Ferry,Airport Gate,Airport Food Court,Airport
1,10,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,4,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gym,Italian Restaurant,Deli / Bodega,Gastropub,Bar
2,13,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,4,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Burger Joint,Restaurant,Mediterranean Restaurant,Café,Men's Store,Pub
3,24,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,4,Bar,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Chinese Restaurant,Gaming Cafe
4,25,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Café,Bakery,Breakfast Spot,Theater,Mexican Restaurant,Restaurant,Health Food Store


### Let's create a map of the cluster using matplotlib

In [29]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_downtown, longitude_downtown], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Let's Examine the 5 Clusters


In [30]:
#cluster 1

downtown_merged.loc[downtown_merged['Cluster Labels'] == 0,downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,-79.360636,0,Coffee Shop,Park,Pub,Café,Bakery,Breakfast Spot,Theater,Mexican Restaurant,Restaurant,Health Food Store
6,M4X,-79.367675,0,Coffee Shop,Park,Restaurant,Café,Pub,Bakery,Pizza Place,Italian Restaurant,Japanese Restaurant,Beer Store
7,M5J,-79.381752,0,Coffee Shop,Aquarium,Hotel,Café,Pizza Place,Italian Restaurant,Brewery,Restaurant,Scenic Lookout,Bakery


In [31]:
#cluster 2

downtown_merged.loc[downtown_merged['Cluster Labels'] == 1,downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M4W,-79.377529,1,Park,Playground,Trail,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store


In [32]:
#cluster 3

downtown_merged.loc[downtown_merged['Cluster Labels'] == 2,downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5V,-79.39442,2,Airport Lounge,Airport Terminal,Airport Service,Boutique,Plane,Sculpture Garden,Boat or Ferry,Airport Gate,Airport Food Court,Airport


In [33]:
#cluster 4

downtown_merged.loc[downtown_merged['Cluster Labels'] == 3,downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M5S,-79.400049,3,Café,Bookstore,Japanese Restaurant,Bar,Bakery,Coffee Shop,Restaurant,Chinese Restaurant,Italian Restaurant,Beer Store


In [34]:
#cluster 5

downtown_merged.loc[downtown_merged['Cluster Labels'] == 4,downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M5K,-79.381576,4,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gym,Italian Restaurant,Deli / Bodega,Gastropub,Bar
2,M4Y,-79.383160,4,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Burger Joint,Restaurant,Mediterranean Restaurant,Café,Men's Store,Pub
3,M5T,-79.400049,4,Bar,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Chinese Restaurant,Gaming Cafe
5,M5B,-79.378937,4,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Ramen Restaurant,Theater,Restaurant,Pizza Place,Tea Room
8,M5C,-79.375418,4,Coffee Shop,Restaurant,Café,Hotel,Clothing Store,Park,Breakfast Spot,Cocktail Bar,Cosmetics Shop,Italian Restaurant
9,M5X,-79.382280,4,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Bar,Bakery,Asian Restaurant,Steakhouse
11,M5L,-79.379817,4,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Steakhouse,Italian Restaurant,Gym,Seafood Restaurant
13,M5H,-79.384568,4,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Clothing Store,Asian Restaurant,Gym,Bakery,Restaurant
14,M5E,-79.373306,4,Coffee Shop,Restaurant,Cocktail Bar,Beer Bar,Seafood Restaurant,Pub,Italian Restaurant,Café,Farmers Market,Bakery


Thanks for Reviewing 